In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
from bs4 import BeautifulSoup
import requests
import html2text

In [ ]:
sel_url = "https://siccode.com/business/"
wiki_url = "https://en.wikipedia.org/wiki/"

company_list = ["LinkedIn","Lynda.com","ZipRecruiter Inc","Indeed.com","SlideShare","Glassdoor","Connectifier"]

company_dict = {}

In [ ]:
# Selennium Portion
driver = webdriver.Chrome('/Users/kkannapp/Downloads/chromedriver')

for company in company_list:
    company_dict[company] = {}
    driver.get(sel_url+re.sub(r'\s+|\.', '-',company))
    try:
        companyCode = driver.find_element_by_xpath("/html/body/div[1]/div/section[2]/div/div/div[2]/div[2]/div/div/a[2]").text
        naics_code = re.findall(r'\d+', companyCode)
        company_dict[company]['naics'] = naics_code[0]
    except:
        company_dict[company]['naics'] = None
    time.sleep(2)

In [ ]:
company_dict

In [ ]:
# Wikipedia Portion
h = html2text.HTML2Text()
h.ignore_links = True

for company in company_list:
    url = requests.get(wiki_url+(re.split(r'\s+',company)[0])).text
    soup = BeautifulSoup(url,'lxml')
    table = soup.find('table',{'class':'infobox vcard'})
    exceptional_row_count = 0
    for tr in table.find_all('tr'):
        if tr.find('th'):
            company_dict[company][h.handle(tr.find('th').text).strip().replace('\n','')] = h.handle(tr.find('td').text).strip().replace('\n',' ')
        else:
            # the first row Logos fall here
            exceptional_row_count += 1
    paragraph = soup.find("div",{'class':'mw-content-ltr'})
    pgtxt = paragraph.find_all('p')[1:3]
    for i in range(len(pgtxt)):
        company_dict[company]['pg_'+str(i)] = h.handle(str(pgtxt[i])).strip().replace('\n',' ')

In [ ]:
company_dict

In [ ]:
import json
with open('result_kk.json', 'w') as fp:
    json.dump(company_dict, fp)